In [ ]:
import urllib.parse as urlparse
import requests
# try-except is a fix for Quantum Mobile release v19.03.0 that does not have requests_cache installed
try:
    import requests_cache
except ImportError:
    pass
import sys
import ipywidgets as ipw
from IPython.lib import backgroundjobs as bg
from app import AppBase
from config import aiidalab_apps
from dulwich.errors import NotGitRepository

In [ ]:
url = urlparse.urlsplit(jupyter_notebook_url)
try:
    name = urlparse.parse_qs(url.query)['app'][0]
except KeyError:
    raise Exception("No app specified")
    exit()

In [ ]:
def update_cache():
    """Run this process asynchronously"""
    requests_cache.install_cache(cache_name='apps_meta', backend='sqlite', expire_after=3600, old_data_on_error=True)
    requests.get(registry_url)
    requests_cache.install_cache(cache_name='apps_meta', backend='sqlite')

# try-except is a fix for Quantum Mobile release v19.03.0 that does not have requests_cache installed
try:
    requests_cache.install_cache(cache_name='apps_meta', backend='sqlite') # at start getting data from cache
    update_cache_background = bg.BackgroundJobFunc(update_cache) # if requests_cache is installed, the
                                                                 # update_cache_background variable will be present
except NameError:
    pass

registry_url = 'https://aiidalab.materialscloud.org/appsdata/apps_meta.json'
try:
    apps = requests.get(registry_url).json()['apps']
    if 'update_cache_background' in globals():
        update_cache_background.start()
except ValueError:
    print("Registry server is unavailable! Can't check for the updates")
    apps = {}

try:
    app_data = apps[name]
except KeyError:
    app_data = None

In [ ]:
app_base = AppBase(name, app_data, aiidalab_apps)

In [ ]:
references = ipw.HTML("""/<a href="start.ipynb">home</a>/<a href="appstore.ipynb">appstore</a>/{}""".format(name))
empty = ipw.HTML('',layout={'width': '400px'} )

In [ ]:
try:
    description = ipw.HTML("""<b> <div style="font-size: 30px; text-align:center;">{}</div></b>
    <br>
    <b>Authors:</b> {}
    <br>
    <b>Description:</b> {}
    <br>
    <b>Git URL:</b> {}""".format(app_base.title,
                          app_base.authors,
                          app_base.description,
                          app_base.git_url))
    logo = app_base.logo
    logo.layout.margin = "100px 0px 0px 0px"
    description.layout = {'width': '800px'}
    display(ipw.VBox([references,
        ipw.HBox([app_base.logo, description]),
        ipw.HBox([app_base.uninstall_button, app_base.update_button, app_base.install_button]),
        ipw.HBox([app_base.install_info])]),
        app_base.version)
except NotGitRepository:
    display(ipw.HTML("""<center><h1>Enable <i class="fa fa-git"></i> first!</h1></center>"""))
    